In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-30 19:04:42--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-30 19:04:43 (13.6 MB/s) - ‘minsearch.py.1’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [8]:
q = 'the course hasalready started, can I still enroll?'

In [9]:
index.fit(documents)

In [13]:
from openai import OpenAI
#import env keys from .env file
import os
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [15]:
client = OpenAI(base_url="https://api.groq.com/openai/v1",
                api_key=os.environ["OPENAI_API_KEY"]
                )

In [17]:
q

'the course hasalready started, can I still enroll?'

In [16]:
model = "llama3-8b-8192"
response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"While it's generally encouraged to enroll in a course before it starts, it's not uncommon for students to join a course after it has already begun. The likelihood of being able to enroll after the course has started depends on several factors, including:\n\n1. **The course's enrollment policies**: Check with the course instructor, institution, or platform hosting the course to see if they have a specific policy on late enrollment. Some might allow it, while others might not.\n2. **Space availability**: If the course is already full, it might not be possible to enroll at this point. Check if there are any available spots.\n3. **Catch-up mechanisms**: If you do manage to enroll after the course has started, check if there are mechanisms in place for catching up on missed material, assignments, or discussions.\n\nTo increase your chances of enrolling in a course after it has started:\n\n1. **Contact the instructor or course administrator**: Reach out to the instructor or course administr

In [18]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [19]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [20]:
def llm(prompt):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [21]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [22]:
rag(query)

'Based on the context, the FAQ database does not provide information on how to run Kafka in general. However, it does provide information on how to run Kafka-related tools and applications, such as Python scripts, Java classes, and Docker images.\n\nThe closest relevant answer is from the "Module 6: streaming with Kafka" section, which mentions running a Java producer/consumer/kstreams/etc in the terminal. The command to run the Java application is:\n\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nHowever, this is specific to a Java application and a project directory, and does not provide general information on how to run Kafka itself.'

In [23]:
rag('the course has already started, can I still enroll?')

'Based on the provided CONTEXT, I can answer the QUESTION:\n\nQUESTION: the course has already started, can I still enroll?\n\nAnswer: Yes, even if the course has already started, you can still enroll.'

In [24]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [25]:
from elasticsearch import Elasticsearch

In [26]:
es_client = Elasticsearch('http://localhost:9200') 

In [38]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "homework-search"

if not es_client.indices.exists(index=index_name):
    es_client.indices.create(index=index_name, body=index_settings)
else:
    print(f"Index '{index_name}' already exists.")

In [29]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [30]:
from tqdm.auto import tqdm

In [31]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [32]:
query = 'I just disovered the course. Can I still join it?'

In [33]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [34]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [35]:
rag(query)

"Based on the context, you can still join the course after the start date. According to the FAQ, you don't need to register, and you're still eligible to submit homeworks."